<a href="https://colab.research.google.com/github/ChoeTaeBin/Machine-Learnig/blob/main/DecisionTreeClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

트리 알고리즘을 분류에 이용할 수 있다. 그리고 성능 평가를 위해 교차 검증 방법을 사용할 수 있다.
하이퍼 파라메터를 바꾸어 가며 교차검증을 하면서 하이퍼 파라메터를 튜닝 할 수 도있으나, GridSearch 또는 RandomSearch를 이용하면 편하다.

In [3]:
#데이터 준비
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

#세트 분할
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

In [4]:
#트리 모델 생성
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)

In [8]:
#일반 교차 검증

from sklearn.model_selection import cross_validate, StratifiedKFold
import numpy as np
splitter = StratifiedKFold(n_splits=10, shuffle = True, random_state = 42) #훈련 세트를 섞기위해 분할기를 만듦
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(scores)
print('평균점수: ', np.mean(scores['test_score']))


{'fit_time': array([0.01029539, 0.00833654, 0.00828481, 0.00799322, 0.00783944,
       0.00870013, 0.00819111, 0.00836301, 0.0080986 , 0.00797558]), 'score_time': array([0.00096536, 0.00088787, 0.00086212, 0.00083733, 0.00085282,
       0.00091791, 0.00092936, 0.00088024, 0.00083661, 0.00086856]), 'test_score': array([0.83461538, 0.87884615, 0.85384615, 0.85384615, 0.84615385,
       0.87307692, 0.85961538, 0.85549133, 0.85163776, 0.86705202])}
평균점수:  0.8574181117533719


하이퍼 파라메터 튜닝

In [10]:
# 그리드 서치
from sklearn.model_selection import GridSearchCV
params = {
          'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2,100, 10)
          } #딕셔너리 형태로 전달

gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1) #cpu모든 코어를 다 쓰겠다는 뜻
gs.fit(train_input, train_target)

print('최적의 parameters: ', gs.best_params_)
dt = gs.best_estimator_ #최적의 모델
print('최적 모델의 점수: ', dt.score(test_input, test_target))

최적의 parameter:  {'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
최적 모델의 점수:  0.8615384615384616


In [11]:
#랜덤 서치
from scipy.stats import uniform, randint
from sklearn.model_selection import RandomizedSearchCV

params = {
    'min_impurity_decrease': uniform(0.0001, 0.001),
    'max_depth': randint(20, 50),
    'min_samples_split': randint(1,25)
}

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

print('최적의 parameters: ', gs.best_params_)
dt = gs.best_estimator_
print('최적 모델의 점수: ', dt.score(test_input, test_target))


최적의 parameters:  {'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628, 'min_samples_split': 15}
최적 모델의 점수:  0.8623076923076923


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 889, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _valida